## Imports / Data Load
- Analyze data to support data modeling and cleaning decisions

In [ ]:
# Imports
from __future__ import annotations
from typing import Tuple, List, Union
from pathlib import Path
import duckdb
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)

# Visuals
import matplotlib.pyplot as plt
import seaborn as sns

# Printing
from rich import print

# Local db path
LOCAL_DB = Path("../data/local_db/ledger.duckdb")

# Connect to DuckDB
con = duckdb.connect(str(LOCAL_DB))

# Load the raw data from the `ledger_raw` table
query = "SELECT * FROM ledger_raw"
df = con.execute(query).df()

# Replace blanks / whitespace with NaN
df.replace(r"^\s*$", np.nan, regex=True, inplace=True)

df.sample(5)

,AGENCYNBR,AGENCYNAME,LEDGER,FISCAL_YEAR,ACCOUNTING_PERIOD,FUND_CODE,FUNDDESCR,CLASS_FLD,CLASSDESCR,DEPTID,DEPTDESCR,ACCOUNT,ACCTDESCR,OPERATING_UNIT,OPERUNITDESCR,PRODUCT,PRODUCTDESCR,PROGRAM_CODE,PGMDESCR,BUDGET_REF,CHARTFIELD1,CF1DESCR,CHARTFIELD2,CF2DESCR,PROJECT_ID,PROJDESCR,POSTED_TOTAL_AMT,ACTIVITY,ACTVDESCR,RESTYPE,RESDESCR,RCAT,RCATDESCR,RSUBCAT,RSUBCATDESCR,ROWID
5085576,29000,EMPLOYMENT SECURITY COMMISSION,ACTUALS,2025.0,3.0,1000,General Fund - No Divisions,40000,Emplmt Security Admw Fund,8800001,Information Technology,513300,Ret.Savings-Def Contr Plan,00016,NaN,NaN,NaN,E0102,NaN,25,NaN,NaN,NaN,NaN,290000000000000,NaN,-54.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AAJsAKAJaAAF+37AAY
3010094,34000,OKLAHOMA STATE DEPARTMENT OF HEALTH,ACTUALS,2024.0,12.0,1000,General Fund - No Divisions,40000,Federal Funds,4020003,IMMUNIZATION,511110,Sals-Regular Pay,CD1P40,NaN,70,NaN,A0100,NaN,24,NaN,NaN,NaN,NaN,BQ2S,NaN,82.48,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2401384,83000,DEPARTMENT OF HUMAN SERVICES,ACTUALS,2024.0,5.0,1000,General Fund - No Divisions,32400,FY14 Human Services Disb Fund,2800007,Adult Protective Serv,513230,Employer Share OPERS,28072700,NaN,NaN,NaN,A0301,NaN,24,NaN,NaN,NaN,NaN,NaN,NaN,3658.96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AAJsAKAI8AANTbSAAu
5340065,29000,EMPLOYMENT SECURITY COMMISSION,ACTUALS,2025.0,7.0,1000,General Fund - No Divisions,40000,Emplmt Security Admw Fund,1000001,Workforce Services,513110,Employer Share-FICA,00071,NaN,NaN,NaN,E0102,NaN,25,NaN,NaN,NaN,NaN,29000000175FY24,NaN,0.24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AAJsAKAJSAALFKQAAS
6462819,56600,DEPARTMENT OF TOURISM AND RECREATION,ACTUALS,2026.0,1.0,1000,General Fund - No Divisions,21500,Ok Tourism Recreation Fund,1020000,Golf Courses,511270,Overtime Wages,NaN,NaN,60,NaN,E0204,NaN,26,NaN,NaN,NaN,NaN,NaN,NaN,224.94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AAJsAKAJvAAKPURAAo


## Null Values
- Columns that are wholly null (not analyzed further):
  - `['PGMDESCR', 'OPERUNITDESCR', 'RSUBCATDESCR', 'RSUBCAT', 'RCATDESCR',
      'RCAT', 'RESDESCR', 'RESTYPE', 'ACTVDESCR', 'CF1DESCR', 'CF2DESCR',
       'PROJDESCR', 'PRODUCTDESCR']`

In [37]:
# Null counts - most columns are empty - drop
null_values = df.isnull().sum().sort_values(ascending=False)
null_values

PGMDESCR             7087897
OPERUNITDESCR        7087897
RSUBCATDESCR         7087897
RSUBCAT              7087897
RCATDESCR            7087897
RCAT                 7087897
RESDESCR             7087897
RESTYPE              7087897
ACTVDESCR            7087897
CF1DESCR             7087897
CF2DESCR             7087897
PROJDESCR            7087897
PRODUCTDESCR         7087897
CHARTFIELD1          6750278
ACTIVITY             6728104
CHARTFIELD2          5896978
PRODUCT              3299609
PROJECT_ID           3246480
OPERATING_UNIT       2412936
BUDGET_REF            649246
PROGRAM_CODE          527056
DEPTDESCR             512411
DEPTID                512411
ROWID                 373357
CLASSDESCR                45
AGENCYNAME                 0
ACCTDESCR                  0
POSTED_TOTAL_AMT           0
ACCOUNT                    0
CLASS_FLD                  0
FUNDDESCR                  0
FUND_CODE                  0
ACCOUNTING_PERIOD          0
FISCAL_YEAR                0
LEDGER        

In [59]:
# Collect columns that are fully null and drop from the data
row_count, _ = df.shape
cols_to_drop = null_values[null_values == row_count].index
df = df.drop(columns=cols_to_drop)
df.sample(5)

,AGENCYNBR,AGENCYNAME,LEDGER,FISCAL_YEAR,ACCOUNTING_PERIOD,FUND_CODE,FUNDDESCR,CLASS_FLD,CLASSDESCR,DEPTID,DEPTDESCR,ACCOUNT,ACCTDESCR,OPERATING_UNIT,PRODUCT,PROGRAM_CODE,BUDGET_REF,CHARTFIELD1,CHARTFIELD2,PROJECT_ID,POSTED_TOTAL_AMT,ACTIVITY,ROWID
860485,9000,OFFICE OF MANAGEMENT AND ENTERPRISE SERV,ACTUALS,2022.0,7.0,1000,General Fund - No Divisions,20000,Revolving Fund,1000026,Agency Business Services,513120,Employer Share-MQFE/FICA,ABS10000,01,D0102,21,NaN,NaN,090S003513,63.71,NaN,AAJsAKAHkAALYYUAAj
4357410,34000,OKLAHOMA STATE DEPARTMENT OF HEALTH,ACTUALS,2024.0,1.0,1000,General Fund - No Divisions,40000,Federal Funds,4040019,Community Health Svcs WIC,513230,Employer Share OPERS,VI0,14,A0000,23,NaN,EA3,301C,12.06,NaN,AAJsAKAI/AAIp/gAAv
3137965,29200,DEPT. OF ENVIRONMENTAL QUALITY,ACTUALS,2024.0,12.0,1000,General Fund - No Divisions,40000,Federal Funds,5500001,Water Quality Div Operational,511110,Sals-Regular Pay,NaN,10,A0102,24,664680000,NaN,292154624,5213.32,24,NaN
4436772,48500,NORTHEASTERN STATE UNIVERSITY,ACTUALS,2024.0,1.0,9000,Higher Educ Component Unit,29000,Educational Gen Operation,1100001,Instruction,521110,In-State Mileage-Motor Vehicle,NaN,10,C0103,23,NaN,NaN,NaN,2112.43,NaN,AAJsAKAI8AANUqEAAh
1409850,30800,STATE BUREAU OF INVESTIGATION,ACTUALS,2023.0,7.0,1000,General Fund - No Divisions,20000,OSBI Revolving Fund,1000040,Investigative Svcs - Fed Grnt,511110,Sals-Regular Pay,699,NaN,B0100,23,NaN,NaN,NaN,18774.37,NaN,AAJsAKAJBAAHQNBAAd


## Data Types
- `FISCAL_YEAR`: Cast to int
- `ACCOUNTING_PERIOD`: Cast to int
- `POSTED_TOTAL_AMT`: Cast to float
- `ACTIVITY`: Understand what this column represents (delta?)

In [60]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 7087897 entries, 0 to 7087896
Data columns (total 23 columns):
 #   Column             Dtype  
---  ------             -----  
 0   AGENCYNBR          int64  
 1   AGENCYNAME         str    
 2   LEDGER             str    
 3   FISCAL_YEAR        float64
 4   ACCOUNTING_PERIOD  float64
 5   FUND_CODE          int64  
 6   FUNDDESCR          str    
 7   CLASS_FLD          str    
 8   CLASSDESCR         str    
 9   DEPTID             str    
 10  DEPTDESCR          str    
 11  ACCOUNT            int64  
 12  ACCTDESCR          str    
 13  OPERATING_UNIT     str    
 14  PRODUCT            str    
 15  PROGRAM_CODE       str    
 16  BUDGET_REF         str    
 17  CHARTFIELD1        str    
 18  CHARTFIELD2        str    
 19  PROJECT_ID         str    
 20  POSTED_TOTAL_AMT   float64
 21  ACTIVITY           str    
 22  ROWID              str    
dtypes: float64(3), int64(3), str(17)
memory usage: 2.4 GB


## Check One-to-One Relationships (b/w ID and Description Columns)
- `Data Modeling`: Separate out `AGENCYNAME` into an `AGENCY` table
- `Data Cleaning`: Standardize `AGENCYNAME` values 
  - (e.g., "OK. SCHOOL OF SCIENCE & MATH." vs. "OK. SCHOOL OF SCIENCE & MATH")

In [95]:
def check_one_to_one_relationship(
    column_pair: Union[Tuple[str, str], List[str]],
    table: str = "LEDGER_RAW"
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Check for one-to-one relationships between two columns in a table.

    Args:
        column_pair (Union[Tuple[str, str], List[str]]): The two columns to check (column1, column2).
        table (str): The name of the table to query. Defaults to "LEDGER_RAW".

    Returns:
        Tuple[pd.DataFrame, pd.DataFrame]: A summary DataFrame and a detailed DataFrame.
    """
    if len(column_pair) != 2:
        raise ValueError("The 'column_pair' parameter must contain exactly two column names.")
    
    column1, column2 = column_pair

    # Define queries
    queries = {
        "summary": f"""
            SELECT
                {column1},
                COUNT(DISTINCT {column2}) AS DISTINCT_{column2}
            FROM {table}
            WHERE {column2} IS NOT NULL
            GROUP BY {column1}
            HAVING DISTINCT_{column2} > 1
        """,
        "details": f"""
            SELECT
                DISTINCT {column1}, {column2}, COUNT(*) AS ROW_COUNT
            FROM {table}
            WHERE {column1} IN (
                SELECT {column1}
                FROM {table}
                WHERE {column2} IS NOT NULL
                GROUP BY {column1}
                HAVING COUNT(DISTINCT {column2}) > 1
            )
            AND {column2} IS NOT NULL
            GROUP BY {column1}, {column2}
            ORDER BY {column1}
        """
    }

    # Execute queries
    summary_df = con.execute(queries["summary"]).df()
    details_df = con.execute(queries["details"]).df()

    if summary_df.empty and details_df.empty:
        print(f"{column_pair=} has 1:1 relationship")
    
    return summary_df, details_df

In [96]:
related_column_pairs = [
    ("AGENCYNBR", "AGENCYNAME"),
    ("FUND_CODE", "FUNDDESCR"),
    ("CLASS_FLD", "CLASSDESCR"),
    ("DEPTID", "DEPTDESCR"),
    ("ACCOUNT", "ACCTDESCR"),
    ("OPERATING_UNIT", "OPERUNITDESCR"),
    ("ACTIVITY", "ACTVDESCR"),
]

for column_pair in related_column_pairs:
    print(f"{column_pair=}")
    summary_df, details_df = check_one_to_one_relationship(
        column_pair
    )
    display(summary_df)
    display(details_df)

column_pair=('AGENCYNBR', 'AGENCYNAME')

,AGENCYNBR,DISTINCT_AGENCYNAME
0,6000,2
1,61900,2
2,62900,2
3,80300,2
4,75500,2
5,47500,2


,AGENCYNBR,AGENCYNAME,ROW_COUNT
0,6000,OK DEP AEROSPACE & AERONAUTICS,4821
1,6000,OKLAHOMA AERONAUTICS COMMISSION,1736
2,47500,OK NEW MOTOR VEHICLE COMM,1437
3,47500,OKLA. MOTOR VEHICLE COMM.,700
4,61900,PHYSICIAN MANPOWER TRNG. COMM.,1181
5,61900,HEALTH CARE WORKFORCE TRNG COM,6284
6,62900,OK. SCHOOL OF SCIENCE & MATH.,7603
7,62900,OK. SCHOOL OF SCIENCE & MATH,1814
8,75500,USED MOTOR VEH & MFG HOUSING,2589
9,75500,USED MOTOR VEHICLE & PARTS,568


column_pair=('FUND_CODE', 'FUNDDESCR')

,FUND_CODE,DISTINCT_FUNDDESCR
0,1830,2


...

,OPERATING_UNIT,OPERUNITDESCR,ROW_COUNT


column_pair=('ACTIVITY', 'ACTVDESCR')

column_pair=('ACTIVITY', 'ACTVDESCR') has 1:1 relationship

,ACTIVITY,DISTINCT_ACTVDESCR


,ACTIVITY,ACTVDESCR,ROW_COUNT
